<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/pull_one_hr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install censusgeocode
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys
from rets import Session
import datetime
from datetime import datetime,timedelta
!pip install pytz
import pytz



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 1.2 MB/s 
     |████████████████████████████████| 62 kB 464 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
username = '0671181_NID'
password = 'Rt$tg6jx'
rets_client = Session(login_url, username, password)
rets_client.login()

#filter = {
       # "MlsStatus": "SLD",
       # "ModificationTimestamp": {"$gte": dt_string}
        #}
#result = rets_client.search(resource='Property', resource_class='Property', limit = 10,search_filter=filter)


tz_IN = pytz.timezone('US/Central')
ini_time_for_now = datetime.now(tz_IN)
today_before_1hours = ini_time_for_now - timedelta(hours = 1)
dt_string = today_before_1hours.strftime("%Y-%m-%dT%H:%M:%S.000")

dt_string


'2022-06-22T10:34:34.000'

In [3]:
#ListingId

ListingId_result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(StatusChangeTimestamp = {}+),(MlsStatus = Act)'.format(dt_string))
lis = []
di ={}
df = pd.DataFrame()
for item in ListingId_result:
  if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence' and item['PreviousStatus'] == "" or item['PreviousStatus'] == "Coming Soon"):
    df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) 
    di = item 


for key in di:
  lis.append(key)

df = df.T
df.columns = lis


df['FullAdress'] = df['StreetNumber'] + " " + df['StreetName']  + df['City'] + ", " +df['StateOrProvince'] + " " + df['PostalCode']
df = df[['FullAdress','ListingId','ListPrice','LivingArea','BathroomsFull','BedroomsTotal','YearBuilt','GarageSpaces','PoolYN','LotSizeAcres']]
df
      


,BathroomsFull,BathroomsHalf,BathroomsTotalDecimal,BathroomsTotalInteger,BedroomsTotal,BuildingAreaTotal,City,CloseDate,ClosePrice,CumulativeDaysOnMarket,...,StreetDirPrefix,StreetDirSuffix,StreetName,StreetNumber,StreetNumberNumeric,StreetSuffix,SubdivisionName,USProperty_MUI,YearBuilt,FullAdress
0,2,0,2.00,2,3,,Fort Worth,,,0,...,,,Dunlavin,11113,11113,Court,Emerald Park Add,402398836,2014,"11113 DunlavinFort Worth, Texas 76052"
1,3,1,3.10,4,5,,Nemo,,,0,...,,,Black Hawk,1401,1401,Trail,Eagles Nest Add,402539440,2018,"1401 Black HawkNemo, Texas 76070"
2,3,1,3.10,4,4,,Irving,,,0,...,,,Picton,7663,7663,Drive,Stonegate,411097792,2020,"7663 PictonIrving, Texas 75063"
3,2,1,2.10,3,4,,Howe,,,0,...,,,Williams,156,156,Trail,Jeremiah A Mcdaniel Abs G0774,406155966,1978,"156 WilliamsHowe, Texas 75459"
4,2,0,2.00,2,4,,Fort Worth,,,0,...,,,Minturn,8436,8436,Drive,Lasater Add,402307988,2003,"8436 MinturnFort Worth, Texas 76131"
5,3,0,3.00,3,3,,Fort Worth,,,0,...,,,Devinstone,2001,2001,Court,Tehama Ridge,402372197,2008,"2001 DevinstoneFort Worth, Texas 76177"
6,2,1,2.10,3,4,,Little Elm,,,0,...,,,Lake Sierra,821,821,Way,Paloma Creek Lakeview Ph 2-,410424605,2017,"821 Lake SierraLittle Elm, Texas 75068"
7,2,0,2.00,2,3,,Balch Springs,,,0,...,,,Cedar Creek,14726,14726,Way,Spring Ridge Ph 2b,410876330,2011,"14726 Cedar CreekBalch Springs, Texas 75180"
8,2,0,2.00,2,3,,Irving,,,6,...,,,Fairbrook,2521,2521,Street,Storyglen North,411082840,1965,"2521 FairbrookIrving, Texas 75062"
9,3,1,3.10,4,4,,Fort Worth,,,0,...,,,Moncrief,9333,9333,Street,Heritage Add,402306858,2004,"9333 MoncriefFort Worth, Texas 76244"


In [4]:
#low_price
lowerPrice_result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(PriceChangeTimestamp = {}+),(MlsStatus = Act)'.format(dt_string))
lis = []
di ={}
df = pd.DataFrame()

for item in lowerPrice_result:
  if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence' and item['PreviousListPrice'] > item['ListPrice']):
    df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) 
    di = item 
for key in di:
  lis.append(key)
df = df.T
df.columns = lis

pre_lis = []
aft_lis = []

df.dropna(subset=['PreviousListPrice'], inplace=True)


for item in df['ListPrice']:
  item = int(round(float(item),0))
  aft_lis.append(item)

for item in df['PreviousListPrice']:
  item = int(round(float(item),0))
  pre_lis.append(item)
  #print(type(item))

df['PreviousListPrice'] = pre_lis
df['ListPrice'] = aft_lis
df['percent'] = (df['PreviousListPrice'] - df['ListPrice'])/ df['ListPrice']

df['FullAddress'] = df['StreetNumber'] + " " + df['StreetName']  + df['City'] + ", " +df['StateOrProvince'] + " " + df['PostalCode']

df = df[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt','FullAddress','percent']]
df.to_csv('low_price.csv')





In [5]:
#Relist - Back on Market

relist_result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(StatusChangeTimestamp = {}+),(MlsStatus = Act)'.format(dt_string))
lis = []
di ={}
df = pd.DataFrame()
# not empty value
for item in relist_result:
    if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence' and item['PreviousStatus']!='Coming Soon' and item['PreviousStatus']!='Active' and item['PreviousStatus']!=""):
      print(item['StatusChangeTimestamp'])
      df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) 
      di = item 

for key in di:
  lis.append(key)
df = df.T
df.columns = lis
print(len(df.index))
df['FullAdress'] = df['StreetNumber'] + " " + df['StreetName'] + df['City'] + ", " +df['StateOrProvince'] + " " + df['PostalCode']
df = df[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt','FullAdress']]
df.to_csv('BOM.csv')

2022-06-22T10:51:53.343
2022-06-22T10:34:52.157
2022-06-22T10:58:25.760
2022-06-22T11:18:10.227
2022-06-22T10:49:15.427
2022-06-22T11:24:49.070
2022-06-22T11:12:59.967
2022-06-22T11:06:23.807
2022-06-22T11:11:20.807
2022-06-22T10:42:46.060
2022-06-22T11:04:41.050
11


In [6]:
# CDOM >=90 Long time no change -ignore now
relist_result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(PriceChangeTimestamp = {}+),(MlsStatus = Act)'.format(dt_string))
lis = []
df = pd.DataFrame()
for item in relist_result:
    if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
      df = df.append(item, ignore_index=True)
df = pd.DataFrame(df)
df['FullAdress'] = df['StreetNumber'] + " " + df['StreetName']  + df['City'] + ", " +df['StateOrProvince'] + " " + df['PostalCode']
df = df[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt','FullAdress']]
print(type(item['ListingContractDate']))
print(type(item['PriceChangeTimestamp']))

#df.to_csv('BOM.csv')

<class 'str'>
<class 'str'>
